In [ ]:
import numpy as np
from IPython.display import display, Math
import logging

for h in logging.getLogger().handlers:
    h.setFormatter(logging.Formatter("[%(levelname)s] %(message)s"))

# Benchmarking an algorithm used to solve a MAX-k-SAT problem

In this notebook we showcase functionality of qubrabench by implementing and benchmarking a hillclimbing algorithm to solve MAX-k-SAT, as described in https://arxiv.org/abs/2203.04975.
<!-- The paper describes two distinct variants of implementing the hillclimber: a simple hillclimber - which uses (quantum) search, and a steep one - which uses (quantum) max finding. -->

## Problem: MAX-k-SAT

Max-k-SAT is a combinatorial optimization problem that given a list of clauses $(C_{i})^{p}_{i=1}$, each a disjunction of at most $k$ literals, and a set of weights $(w_{i})^{p}_{i=1}$, asks us to maximize the weight of the satisfied clauses,
$$\varphi(x) := \sum ^{p} _{i=1} w_{i}C_{i}(x),$$
over all assignments $x \in \{0, 1\}^{q}$ of the variables. This problem is known to be NP-hard for $k ≥ 2$.

In [ ]:
from typing import Sequence
from dataclasses import dataclass


@dataclass
class Clause:
    terms: Sequence[int]
    weight: int

    def is_satisfied(self, assignment: Sequence[bool]) -> bool:
        return all(assignment[abs(x) - 1] == (x > 0) for x in self.terms)


@dataclass
class Instance:
    n_var: int
    clauses: Sequence[Clause]

    def evaluate_weight_of_assignment(self, assignment: Sequence[bool]) -> int:
        """compute \varphi(assignment)"""
        return sum(
            clause.weight for clause in self.clauses if clause.is_satisfied(assignment)
        )

### Example Instance

Consider the following instance for MAX-3-SAT, with 4 variables $x = (x_{1}, x_{2}, x_{3}, x_{4})$: 

$$(\neg x_{1} \lor \neg x_{2} \lor x_{3})_{3} \land (x_{1} \lor x_{2} \lor \neg x_{4})_{5} \land (\neg x_{1} \lor \neg x_{2} \lor x_{4})_{1}$$

In the course of the following sections, we will build an algorithm for MAX-$k$-SAT and use this instance to demonstrate how it works.

In [ ]:
demo_inst = Instance(
    n_var=4,
    clauses=[
        Clause(terms=[-1, -2, 3], weight=3),
        Clause(terms=[1, 2, -4], weight=5),
        Clause(terms=[-1, -2, 4], weight=1),
    ],
)

In this representation, we store the index $i$ for each variable $x_i$ in the clause. A negated literal $\neg x_i$ is stored as $-i$.

We can evaluate $\varphi(x)$ for a given $x$ as follows:

In [ ]:
print(demo_inst.evaluate_weight_of_assignment((1, 1, 0, 0)))

## Algorithm: Hillclimber

<!-- A known algorithm to solve this problem is the hillclimber algorithm, which we will be implementing in this notebook. -->
We start with a random assignment $x \in \{0, 1\}^{n}$.
We then repeatedly look for improvements (higher values) of $\varphi$ in the _neighbourhood_ $N(x)$ of the current assignment $x$, and pick one if it exists (i.e. set $x$ to the chosen neighbour, and repeat).
Here, $N(x)$ is the set of all assignments that differ from $x$ in exactly one bit.

We can have different variants of the hillclimber based on how we pick a _better_ neighbour.
In this notebook, we discuss the **steep** hillclimber: pick the neighbour $y$ with the highest value of $\varphi(y)$.
$$x' = \arg\max_{y \in N(x)}(\varphi(y))$$
If $\varphi(x') > \varphi(x)$, we set $x \leftarrow x'$ and repeat, otherwise we stop.

<!-- ## Solving our problem instance

As mentioned above, we can use a hillclimber algorithm to solve the MAX-k-SAT problem. For the purpose of simplicity within this notebook, we will be focussing on the steep hillclimber. Side note: This repository also contains a [hillclimber algorithm](https://github.com/qubrabench/qubrabench/blob/notebooks/examples/sat/bench_hillclimber.py), which uses a more advanced approach and has much better performance than our approach. However, as we are focussing on the functionality of qubrabench in this notebook, this more complex implementation will not be covered in more detail.
 -->
 <!-- 
We will assume, that $d=1$, which enables us to simplify some of our functions. This means, when we look at the neighbourhood of a given variable assignment $y$, we will always be looking at assignments $z\in N_{1}(y)$, that differ from our current assignment in exactly one place. -->

To get our starting assignment, we implement a function which randomly generates a sequence of zeros and ones. 

In [ ]:
def generate_random_assignment(n_var: int, *, rng=np.random.default_rng(42)):
    return tuple(rng.choice((0, 1), size=n_var))


random_assignment = generate_random_assignment(demo_inst.n_var)
print(f"Random assignment: {random_assignment}, which means:")

Math(
    ", ".join(
        f"x_{i + 1} = {random_assignment[i]}" for i, x_i in enumerate(random_assignment)
    )
)

We now need to compute the neighbourhood set $N(x)$ for a given assignment $x$.

In [ ]:
def get_neighbours(x):
    neighbours = []
    for i, x_i in enumerate(x):
        y = list(x)
        y[i] = 1 - x_i
        neighbours.append(tuple(y))
    return neighbours


print(get_neighbours((0, 0, 0, 0)))

We can now pick the _best_ neighbour for a given assignment $x$:

In [ ]:
def find_best_neighbour(x: Sequence[bool], inst: Instance):
    return max(
        get_neighbours(x),
        key=inst.evaluate_weight_of_assignment,
    )


best_neighbour = find_best_neighbour((0, 0, 0, 0), demo_inst)
neighbour_weight = demo_inst.evaluate_weight_of_assignment(best_neighbour)

print(f"Best neighbour: {best_neighbour}, weight = {neighbour_weight}")

With the above subroutines defined, we can now implement the hillclimber algorithm.

In [ ]:
def hill_climber_sat(
    inst: Instance,
    *,
    rng=np.random.default_rng(42),
):
    # starting assignment
    x = generate_random_assignment(inst.n_var, rng=rng)
    weight = inst.evaluate_weight_of_assignment(x)
    logging.info(f"Starting with assignment: {x}, with weight {weight}")

    while True:
        x_new = find_best_neighbour(x, inst)
        neighbour_weight = inst.evaluate_weight_of_assignment(x_new)
        if neighbour_weight > weight:
            x, weight = x_new, neighbour_weight
            logging.info(f"Found better neighbour: {x}, with weight {weight}")
        else:
            break

    logging.info(f"Completed. Optimal assignment: {x}. Weight: {weight}")
    return x


logging.getLogger().setLevel(logging.INFO)
hill_climber_sat(demo_inst)

## Spotting a pattern: Max

If we take another look at this approach, we will notice, that we calculate the maximum value of a set of numbers within the find_better_assignment function. In this case the maximum weight amongst the weights of all neighbours using the Python max function. 

As it happens there is also a very well known max function, which runs on quantum computers, as described in the paper mentioned in the introduction. With this in mind, the question naturally occurs, what would happen if we ran the computation of max on a quantum machine using this quantum max function and if this would have any performance improvements.

This library offers a function, which enables a user to simulate doing exactly this and to keep track of runtime stats and approximations for quantum call counts, if the quantum max function was used and run on a quantum machine.

## Max Function

The library includes a set of common functions you would need in various circumstances, for example the search and the max function. These return the result you would expect from other implementations of these functions, and at the same time add in quantum benchmarking functionality. This simultaneously makes it easier to understand the use-cases of the library and makes it relatively easy to adapt existing code to use quantum benchmarking.

When we talk about quantum benchmarking here, we mean approximating the amount of calls needed to execute the function with identical parameters on a quantum computer. This library achieves this by counting the calls run on the classical (in this case most likely your) machine and basing performance assumptions on these measurements. All provided functions in qubrabench currently require a stats object, in which these collected statistics and approximations are stored.

Let's take a look at the docstrings of the function we will need to solve our problem - max.

In [ ]:
from qubrabench.algorithms.max import max as qmax

%pdoc qmax

As you will notice, the arguments for this max function are very similar to the Python max function. Indeed, one could run this function instead of max with the same parameters and would still get the wanted functionality. However, it is also possible to supply this function with two additional arguments - error and stats - which enables us to keep track of the calculated statistics.

To demonstrate this, we will rewrite the find_better_assignment function from above to use the qubrabench max function. This is simply done by importing this function with the name max. We also create a stats object as provided by qubrabench and pass this to the max function together with a value for the error argument. Now we can run the same hill_climber_sat function we defined above and take a look at the stats object after it ran using the redefined find_better_assignment function.

In [ ]:
from qubrabench.algorithms.max import max
from qubrabench.stats import QueryStats


stats = QueryStats()


def find_best_neighbour(x, inst: Instance):
    return max(
        get_neighbours(x),
        key=inst.evaluate_weight_of_assignment,
        error=10**-5,  # new parameter!
        stats=stats,  # new parameter!
    )


hill_climber_sat(demo_inst)

print()
print(f"Classical Actual Queries: {stats.classical_actual_queries}")
print(f"Expected Quantum Queries: {stats.quantum_expected_quantum_queries}")

As you can see in this example, the effort to use the qubrabench max function was very minimal and hardly invasive in our code. At the same time, it enabled us to gain valuable insight into our query statistics regarding the use of a quantum algorithm. As we can see, using the quantum max algorithm would worsen our runtime for this small problem instance. However, there are cases in which there are expected benefits for the runtime. These only occur in very large problem instances, which would take very long to run with our hillclimber implementation.

In the following section we will be running the hillclimber provided in this library, because it runs much faster for larger problem instances. This will nonetheless be sufficient to demonstrate how these benchmarking results may look.

## Advanced Benchmarking

The imported `run` function executes the hillclimber on a random instance and returns the statistics in a pandas DataFrame.

In [ ]:
import pandas as pd
from typing import Optional
from dataclasses import asdict


def run(
    k: int,
    n: int,
    r: int = 3,
    *,
    n_runs: int,
    rng: np.random.Generator,
    error: Optional[float] = None,
    steep: bool = False,
) -> pd.DataFrame:
    assert n_runs >= 1
    assert 3 <= r <= 6

    clause_count = n * r

    history = []
    for run_ix in range(n_runs):
        logging.info(f"Instance: k={k}, r={r}, n={n}, steep={steep}, #{run_ix}")

        global stats
        stats = QueryStats()  # reset

        # Generate random problem instance
        inst = Instance(
            n_var=n,
            clauses=[
                Clause(
                    terms=rng.choice([-1, 1])
                    * rng.integers(1, n, endpoint=True, size=k),
                    weight=rng.integers(1, 21),
                )
                for _ in range(clause_count)
            ],
        )

        hill_climber_sat(inst)

        # save record to history
        rec = asdict(stats)
        rec["n"] = n
        rec["r"] = r
        rec["k"] = k
        history.append(rec)

    # return pandas dataframe
    df = pd.DataFrame(
        [list(row.values()) for row in history], columns=list(history[0].keys())
    )
    return df


logging.getLogger().setLevel(logging.WARN)
run(3, 10, 3, n_runs=5, rng=np.random.default_rng(seed=100))

With this function, we can run the hillclimber multiple times and then compute the average of the results.

Let's run the steep hill climber for $n = 100$, $n = 300$ and $n=500$. We will run the hill climber five times for each $n$ and consistently use $k = 3$.

In [ ]:
%%time
data_100 = run(
    k=3,
    r=3,
    n=100,
    n_runs=5,
    rng=np.random.default_rng(seed=100),
    error=10**-5,
    steep=True,
)
data_100

In [ ]:
%%time
data_300 = run(
    k=3,
    r=3,
    n=300,
    n_runs=5,
    rng=np.random.default_rng(seed=100),
    error=10**-5,
    steep=True,
)
data_300

In [ ]:
%%time
data_500 = run(
    k=3,
    r=3,
    n=500,
    n_runs=5,
    rng=np.random.default_rng(seed=100),
    error=10**-5,
    steep=True,
)
data_500

As you can see in the resulting tables, for problem instances with $n \le 500$ we can still achieve no performance improvement in using the quantum max algorithm. However, this would change at some point with $n>10^5$. The exact point for the steep hillclimber has not been calculated in the paper and it would take quite a while to do so, however for the simple hillclimber we see performance benefits around $n=10^4$. 

The values mentioned above are gained from the plots provided in the paper (page 28). Plots are a good way to visualize this kind of data, which is why qubrabench provides some functionality, which is supposed to make plotting these results easier. 

## Plotting
After we have benchmarked the solving of a couple of our problem instances, we can take a look at the plotting functionality qubrabench provides. We use the `PlottingStrategy` wrapper to define our plot parameters and configuration.

In [ ]:
from qubrabench.utils.plotting_strategy import PlottingStrategy


class Plotter(PlottingStrategy):
    def __init__(self):
        # TODO: explain
        self.colors[""] = "blue"

    def get_plot_group_column_names(self):
        return ["k", "r"]

    def get_data_group_column_names(self):
        """
        Generate a data line for each unique value in the specified columns.
        Useful if you the data was generated with different tags based on implementation source, parameter choice etc., that one wants to compare against in a single plot.

        Example: ["impl"] - a line will be generated for each unique `impl` label.
        """
        return []

    def compute_aggregates(self, data, *, quantum_factor):
        # compute combined query costs of quantum search
        c = data["quantum_expected_classical_queries"]
        q = data["quantum_expected_quantum_queries"]
        data["quantum_cost"] = c + quantum_factor * q
        return data

    def x_axis_column(self):
        return "n"

    def x_axis_label(self):
        return "$n$"

    def y_axis_label(self):
        return "Queries"

    def get_column_names_to_plot(self):
        return {
            "classical_actual_queries": ("Classical Queries", "o"),
            "quantum_cost": ("Quantum Queries", "x"),
        }

We can put all the benchmark stats in a single table and run the plotter. 

In [ ]:
%pdoc Plotter.plot

In [ ]:
data = pd.concat([data_100, data_300, data_500])
Plotter().plot(data, quantum_factor=2, y_lower_lim=10)

Now we can also run the "simple" hillclimber for the above instance sizes, and compare the two benchmarks. We use the library function provided by qubrabench here, as we have not implemented the simple hillclimber here.

In [ ]:
from hillclimber import run as run_lib

In [ ]:
%%time
data_simple = [
    run_lib(
        k=3,
        r=3,
        n=n,
        n_runs=5,
        rng=np.random.default_rng(seed=100),
        error=10**-5,
        steep=False,
    )
    for n in [100, 300, 500]
]
data_simple = pd.concat(data_simple)

In [ ]:
# add an extra column to distinguish the source (i.e. type of hillclimb)
full_data = []
for d, is_steep in [(data, True), (data_simple, False)]:
    d = d.copy()
    d.insert(0, "steep", is_steep)
    full_data.append(d)
full_data = pd.concat(full_data)

We modify the above plotter a bit as we now want to group the data by column "steep" (in the same plot).

In [ ]:
class FullPlotter(Plotter):
    def __init__(self):
        self.colors["steep = False"] = "red"
        self.colors["steep = True"] = "blue"

    def get_data_group_column_names(self):
        return ["steep"]


FullPlotter().plot(full_data, quantum_factor=2, y_lower_lim=10)

As we can see here, we need less queries for the simple hillclimber for our problem instances. The simple hillclimber also shows more apparent possibilities for performance improvements through quantum algorithms, specifically Grover search in that case. This can be seen even more clearly in the plots provided in this paper on page 28: https://arxiv.org/abs/2203.04975